In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명을 해주세요")
parser = StrOutputParser()
chain = prompt | model | parser

print(chain.invoke({"topic":"더블딥"}))

"더블딥"은 경제학에서 사용되는 용어로, 경기 침체가 한번 발생한 후 잠시 회복세를 보이다가 다시 침체로 돌아가는 현상을 설명합니다. 이 경우, 경제는 두 번의 저점(low point)을 경험하게 되며, 그런 모습이 마치 "W" 자 형태를 이루는 것에서 이름이 유래했습니다. 더블딥은 일반적으로 소비자 신뢰의 부족, 투자 감소, 그리고 외부 충격 등 여러 요인에 의해 발생할 수 있습니다. 이 현상은 경제 회복의 불확실성을 강조하며, 정책 입안자들에게 큰 도전 과제가 됩니다.


In [5]:
chain.batch([{"topic":"더블딥"}, {"topic":"인플레이션"}])

['더블딥(Double Dip)은 경제학에서 사용하는 용어로, 경기 침체가 한 번 끝난 후 다시 경제가 후퇴하는 상황을 나타냅니다. 즉, 경제가 일시적으로 회복된 것처럼 보이다가 다시 어려워지는 경우를 의미합니다. 일반적으로 통계적으로 두 개의 별도의 침체 기간이 존재하며, 이로 인해 경제적인 불확실성이 커질 수 있습니다. 더블딥은 주로 금융위기나 대규모 경제 위기 이후에 발생할 수 있습니다.',
 '인플레이션은 경제 전반에서 물가가 지속적으로 상승하는 현상을 의미합니다. 물가가 오르면 화폐의 구매력이 감소하게 되어, 동일한 금액으로 구입할 수 있는 재화와 서비스의 양이 줄어듭니다. 인플레이션은 여러 요인에 의해 발생할 수 있으며, 주로 수요 증가, 생산비 상승, 통화 공급의 증가 등이 원인이 됩니다. 인플레이션은 적정 수준에서 경제 성장에 긍정적인 영향을 미칠 수 있지만, 지나치게 높은 인플레이션은 경제에 부정적인 영향을 미칠 수 있습니다. 일반적으로 중앙은행은 통화정책을 통해 인플레이션을 조절하려고 합니다.']

In [6]:
for token in chain.stream({"topic":"더블딥"}):
    print(token, end="", flush=True)

"더블딥(Double Dip)"은 경제학에서 두 번의 경기 침체를 의미하는 용어입니다. 이 상황에서는 한 번의 침체가 끝난 후에 경제가 빠르게 회복되지 않고 다시 또 다른 침체에 들어가는 경우를 말합니다. 예를 들어, 2000년대 초반의 미국 경제나 2008년 금융 위기 이후의 경기 회복 과정에서 일부 전문가들이 더블딥 가능성을 언급했습니다. 이런 현상은 소비자 신뢰도 저하, 고용 불안정 등으로 인해 발생할 수 있습니다.

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해주세요: {answer}")

composed_chain = {"answer": chain} | analysis_prompt | model | StrOutputParser()

composed_chain.invoke({"topic":"더블딥"})

'"Double dip" is a term used in economics to describe the phenomenon where a recession occurs, followed by a temporary recovery, and then a return to recession. This concept contrasts with V-shaped recovery (rapid rise) and U-shaped recovery (gradual rise). When a double dip occurs, it indicates that consumption and investment recoveries do not sustain and instead decline again. This phenomenon can undermine confidence in economic recovery and increase economic uncertainty.'

In [10]:
model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template("{topic}에 대해 짧게 설명해주세요.")
chain = prompt | model | StrOutputParser()
analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 번역해주세요: {answer}")

# 람다를 사용한 경우 스트리밍 작업과 호환되지 않ㅇ르 수 있음
composed_chain_with_lambda = (
    chain
    | (lambda input: {"answer": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)
composed_chain_with_lambda.invoke({"topic":"더블딥"})

'A "double-dip" is a term used to describe a situation where an economy begins to recover but then falls back into recession. It generally refers to a scenario where, after emerging from a recession and starting to grow, the economy experiences another downturn due to unexpected factors or structural problems. This phenomenon can negatively impact investment, employment, and production, and poses challenges for policymakers. A double-dip is a closely watched indicator in economic and financial markets.'

In [11]:
composed_chain_with_pipe = (
    chain
    .pipe(lambda input: {"answer": input})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)
composed_chain_with_pipe.invoke({"topic":"더블딥"})

'Double Dip refers to a phenomenon in the economy where, after a recovery, a recession occurs again. This situation arises when the economy seems to emerge from a downturn and recover, only to quickly fall back into recession. Such occurrences can be caused by a variety of factors, including ineffective economic policies, external shocks, and a decrease in consumption and investment. They often present significant challenges for investors and policymakers, necessitating careful economic analysis and response.'

In [15]:
composed_chain_with_pipe = chain.pipe(lambda input: {"answer": input}, analysis_prompt, model, StrOutputParser())

composed_chain_with_pipe.invoke({"topic":"더블딥"})

'The term "double dip" is commonly used in economics to describe a situation where an economy falls back into recession after initially recovering. In other words, it refers to a phenomenon where the economy experiences a recession, shows signs of recovery, but then declines again into another recession. On a graph, this pattern resembles a "double bottom," which is why it is called a "double dip." Such occurrences can be triggered by various factors, such as changes in economic policy or external shocks, and they increase uncertainty and instability in the economy.'

In [16]:
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(model="gpt-4o")
kor_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 설명해주세요.")
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template("{topic}에 대해 짧게 영어로 설명해주세요.")
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = parallel_chain.invoke({"topic":"더블딥"})
print("한글 설명:\n", result["kor"])
print("영어 설명:\n", result["eng"])

한글 설명:
 더블딥(Double Dip)이란 일반적으로 경제나 금융 분야에서 사용되는 용어로, 경제가 회복세를 보이다가 다시 침체에 빠지는 상황을 설명할 때 사용됩니다. 예를 들어, 경기침체가 끝나고 경제가 성장세를 보이다가, 예상치 못한 요인들로 인해 다시 경기침체를 겪게 되는 것을 의미합니다. 이러한 상황은 소비자 신뢰 하락, 투자 감소, 글로벌 경제 불안정성 등 다양한 이유로 발생할 수 있습니다. 더블딥 현상이 발생하면 경제 회복이 지연되고 추가적인 정책 대응이 필요할 수 있습니다.
영어 설명:
 "Double dip" refers to a situation where an economy experiences a recession, recovers briefly, and then falls back into another recession. This pattern of economic activity can be visualized as a "W"-shaped curve on a graph, with each "dip" representing a period of economic decline.
